In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas  as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pickle
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
# Read dataset and remove redundant columns
data = pd.read_csv("expert1.csv")
del data["post_created"]
del data["group_id"]
del data["x"]
data

id                                              value risk
0    167  Zapraszamy wszystkich na Kebaba pod Galerie Gr...  nie
1    168  Zapraszamy na wieczór z retro muzyką na żywo!W...  nie
2    169  Otwarcie sezonu w ADRENALINA PARK!Szukasz niez...  nie
3    170  Zapraszam na MENSCAPE- 4-dniowy warsztat pracy...  nie
4    171  Nasi kierowcy nie tylko zawiozą Was do celu al...  nie
..   ...                                                ...  ...
407  574  Realizujemy dowozy, darmowe od 40 zł na tereni...  nie
408  575  Zapraszamy wszystkich na Kebaba pod Galerie Gr...  nie
409  576  Zapraszamy na wieczór z retro muzyką na żywo!W...  nie
410  577  Otwarcie sezonu w ADRENALINA PARK!Szukasz niez...  nie
411  578  Zapraszam na MENSCAPE- 4-dniowy warsztat pracy...  nie

[412 rows x 3 columns]

In [3]:
# Labels cleaning
data['risk'] = data['risk'].str.lower()
data["risk"].replace({"nie": 0, "nie.": 0, "tak.": 1, "tak": 1}, inplace=True)
data["risk"].value_counts()

0    373
1     39
Name: risk, dtype: int64

In [4]:
# Clean empty values, make text lower 
y = data["risk"]
data["value"] = data["value"].fillna('')
data['value'] = data['value'].str.lower()
data["value"]

0      zapraszamy wszystkich na kebaba pod galerie gr...
1      zapraszamy na wieczór z retro muzyką na żywo!w...
2      otwarcie sezonu w adrenalina park!szukasz niez...
3      zapraszam na menscape- 4-dniowy warsztat pracy...
4      nasi kierowcy nie tylko zawiozą was do celu al...
                             ...                        
407    realizujemy dowozy, darmowe od 40 zł na tereni...
408    zapraszamy wszystkich na kebaba pod galerie gr...
409    zapraszamy na wieczór z retro muzyką na żywo!w...
410    otwarcie sezonu w adrenalina park!szukasz niez...
411    zapraszam na menscape- 4-dniowy warsztat pracy...
Name: value, Length: 412, dtype: object

In [5]:
# Load polish stopwords
x = requests.get('https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt').text.split("\n")
def clean_stopwords(ss):
    for word in x:
        begin = f"{word} "
        if ss.startswith(begin):
            ss.replace(begin, "", 1) 
        in_middle = f" {word} "
        if in_middle in ss:
            x.replace(in_middle, "")   
    return ss

# Clean posts from polish stop word
data['value'].apply(lambda x: [clean_stopwords(item) for item in x])
data["value"]

0      zapraszamy wszystkich na kebaba pod galerie gr...
1      zapraszamy na wieczór z retro muzyką na żywo!w...
2      otwarcie sezonu w adrenalina park!szukasz niez...
3      zapraszam na menscape- 4-dniowy warsztat pracy...
4      nasi kierowcy nie tylko zawiozą was do celu al...
                             ...                        
407    realizujemy dowozy, darmowe od 40 zł na tereni...
408    zapraszamy wszystkich na kebaba pod galerie gr...
409    zapraszamy na wieczór z retro muzyką na żywo!w...
410    otwarcie sezonu w adrenalina park!szukasz niez...
411    zapraszam na menscape- 4-dniowy warsztat pracy...
Name: value, Length: 412, dtype: object

In [7]:
# Vectorize posts values
cc = CountVectorizer()
X = cc.fit_transform(data['value'])
X.shape 

(412, 3463)

In [8]:
X = pd.DataFrame(X.todense(), columns=cc.get_feature_names_out())
X

00  000  007  009  009krzyż  00rzeszów  00warszawska  01  016  02  ...  \
0     0    0    0    0         0          0             0   0    0   0  ...   
1     0    0    0    0         0          0             0   0    0   0  ...   
2     0    0    0    0         0          0             0   0    1   0  ...   
3     0    0    0    0         0          0             0   0    0   0  ...   
4     0    0    0    0         0          0             0   0    0   0  ...   
..   ..  ...  ...  ...       ...        ...           ...  ..  ...  ..  ...   
407   0    0    0    0         0          0             0   0    0   0  ...   
408   0    0    0    0         0          0             0   0    0   0  ...   
409   0    0    0    0         0          0             0   0    0   0  ...   
410   0    0    0    0         0          0             0   0    0   0  ...   
411   0    0    0    0         0          0             0   0    0   0  ...   

     żona  życie  życiu  życzymy  żyje  żywe  żywicy  żywo  żywy  żyłkę  
0       0      0      0        0     0     0       0     0     0      0  
1       0      0      0        0     0     0       0     1     0      0  
2       0      0      0        0     0     0       0     0     0      0  
3       0      0      0        0     0     0       0     0     0      0  
4       0      0      0        0     0     0       0     0     0      0  
..    ...    ...    ...      ...   ...   ...     ...   ...   ...    ...  
407     0      0      0        0     0     0       0     0     0      0  
408     0      0      0        0     0     0       0     0     0      0  
409     0      0      0        0     0     0       0     1     0      0  
410     0      0      0        0     0     0       0     0     0      0  
411     0      0      0        0     0     0       0     0     0      0  

[412 rows x 3463 columns]

In [10]:
# Split data 1:1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=111)

In [15]:
# Test single ML model and save output 
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
dtc.fit(X_train, y_train)

pred = dtc.predict(X_test)
accuracy_score(y_test , pred)

test  = pd.DataFrame()
test["Yreal"] = y_test
test["Ypred"] = pred
test.to_csv("test.csv")

# Example of saving model for future
with open('model.pkl','wb') as f:
    pickle.dump(dtc,f)
obj = pd.read_pickle('model.pkl')


In [16]:
# Use librabry with many ML models
import lazypredict
from lazypredict.Supervised import LazyClassifier    

In [17]:
# Test many ML models automatically
clf = LazyClassifier(verbose=0,ignore_warnings=True)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models.sort_values(by='Model')

100%|██████████| 29/29 [00:13<00:00,  2.18it/s]


Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
AdaBoostClassifier                 0.93               0.56     0.56      0.90   
BaggingClassifier                  0.93               0.56     0.56      0.90   
BernoulliNB                        0.88               0.48     0.48      0.86   
CalibratedClassifierCV             0.92               0.50     0.50      0.88   
DecisionTreeClassifier             0.93               0.56     0.56      0.90   
DummyClassifier                    0.92               0.50     0.50      0.88   
ExtraTreeClassifier                0.94               0.67     0.67      0.93   
ExtraTreesClassifier               0.93               0.56     0.56      0.90   
GaussianNB                         0.94               0.81     0.81      0.94   
KNeighborsClassifier               0.92               0.50     0.50      0.88   
LGBMClassifier                     0.92               0.50     0.50      0.88   
LabelPropagation                   0.92               0.53     0.53      0.89   
LabelSpreading                     0.92               0.53     0.53      0.89   
LinearDiscriminantAnalysis         0.92               0.53     0.53      0.89   
LinearSVC                          0.90               0.52     0.52      0.88   
LogisticRegression                 0.92               0.53     0.53      0.89   
NearestCentroid                    0.92               0.50     0.50      0.88   
PassiveAggressiveClassifier        0.92               0.55     0.55      0.89   
Perceptron                         0.95               0.73     0.73      0.94   
QuadraticDiscriminantAnalysis      0.92               0.50     0.50      0.88   
RandomForestClassifier             0.92               0.53     0.53      0.89   
RidgeClassifier                    0.92               0.53     0.53      0.89   
RidgeClassifierCV                  0.92               0.53     0.53      0.89   
SGDClassifier                      0.91               0.52     0.52      0.88   
SVC                                0.92               0.50     0.50      0.88   
XGBClassifier                      0.92               0.50     0.50      0.88   

                               Time Taken  
Model                                      
AdaBoostClassifier                   0.86  
BaggingClassifier                    0.41  
BernoulliNB                          0.18  
CalibratedClassifierCV               4.20  
DecisionTreeClassifier               0.23  
DummyClassifier                      0.14  
ExtraTreeClassifier                  0.18  
ExtraTreesClassifier                 0.54  
GaussianNB                           0.24  
KNeighborsClassifier                 0.22  
LGBMClassifier                       0.33  
LabelPropagation                     0.21  
LabelSpreading                       0.16  
LinearDiscriminantAnalysis           0.37  
LinearSVC                            1.09  
LogisticRegression                   0.28  
NearestCentroid                      0.18  
PassiveAggressiveClassifier          0.22  
Perceptron                           0.17  
QuadraticDiscriminantAnalysis        0.31  
RandomForestClassifier               0.42  
RidgeClassifier                      0.23  
RidgeClassifierCV                    0.21  
SGDClassifier                        0.20  
SVC                                  0.28  
XGBClassifier                        0.61